## Real-Time Sound Monitoring System

This notebook demonstrates a **real-time sound level monitoring system** using Python. It captures audio from the microphone and computes the **sound pressure level** (SPL) in decibels (dB). The system categorizes the sound level into three distinct thresholds:

- **Safe**: Sound levels below 70 dB, which are considered safe for prolonged exposure.
- **Moderate**: Sound levels between 70 and 85 dB, indicating moderate sound levels that may require caution.
- **Dangerous**: Sound levels above 85 dB, where exposure may be harmful over extended periods.

The system works as follows:

1. **Audio Capture**: The microphone stream is captured in real-time using the `sounddevice` library.
2. **Audio Processing**: The captured audio chunks are processed to calculate the **Root Mean Square (RMS)**, which is then converted into decibels (dB).
3. **Sound Level Categorization**: The calculated decibel levels are categorized into different sound level ranges.
4. **Output**: The decibel level and its corresponding status are printed in real-time every second, along with warnings if the sound level exceeds defined thresholds.

This implementation runs for a duration of **1 minute** and continuously monitors the sound levels during that period.

### Key Parameters:
- **Sampling Rate**: 44100 Hz (standard for high-quality audio)
- **Threshold Levels**: 
    - Safe: Below 70 dB
    - Moderate: 70–85 dB
    - Dangerous: Above 85 dB
- **Monitoring Duration**: 60 seconds (1 minute)

This system can be useful for monitoring sound environments in real-time, ensuring user safety, and providing warnings for potentially harmful exposure to high sound levels.


In [1]:
import numpy as np
import sounddevice as sd
import time
import queue

# Parameters for the sound monitoring
SAMPLERATE = 44100  # Sampling rate in Hz (how many samples per second)
CHUNK_DURATION = 1  # Duration of each audio chunk in seconds
THRESHOLD_SAFE = 70  # Safe sound level threshold in decibels (dB)
THRESHOLD_MODERATE = 85  # Moderate sound level threshold in decibels (dB)
MONITORING_DURATION = 60  # Total monitoring duration in seconds (1 minute)

# Queue to handle the audio data from the stream
audio_queue = queue.Queue()

# Callback function to fetch audio data from the microphone
def audio_callback(indata, frames, time, status):
    """This function is called for each audio chunk from the microphone stream."""
    if status:
        print(status)  # Print any status messages (e.g., buffer underrun errors)
    audio_queue.put(indata.copy())  # Copy the incoming audio data to the queue for processing

# Function to calculate decibel levels from the audio signal
def calculate_db(audio_chunk):
    """Convert the audio chunk to decibels (dB)."""
    rms = np.sqrt(np.mean(audio_chunk**2))  # Calculate the Root Mean Square (RMS) of the audio signal
    db = 20 * np.log10(rms + 1e-6)  # Convert RMS to decibels, adding a small constant to avoid log(0)
    return db

# Function for monitoring the sound quality in real-time
def monitor_sound_quality():
    """Monitor the sound levels for 1 minute and print warnings based on thresholds."""
    print("Monitoring sound levels for 1 minute...")

    start_time = time.time()  # Record the start time for the monitoring session

    try:
        # Open an audio stream for real-time monitoring
        with sd.InputStream(
            samplerate=SAMPLERATE, 
            channels=1,  # Mono audio (1 channel)
            blocksize=int(SAMPLERATE * CHUNK_DURATION),  # Number of frames per chunk (audio chunk size)
            callback=audio_callback  # The callback function that processes the audio data
        ):
            while True:
                current_time = time.time()  # Get the current time
                elapsed_time = current_time - start_time  # Calculate the elapsed time

                if elapsed_time >= MONITORING_DURATION:
                    print("\nMonitoring complete. Time limit reached.")  # End the monitoring session
                    break

                try:
                    # Get the next audio chunk from the queue
                    audio_chunk = audio_queue.get(timeout=1)
                    audio_chunk = audio_chunk.flatten()  # Flatten the audio chunk to 1D array for processing

                    # Calculate the decibel level of the audio chunk
                    db_level = calculate_db(audio_chunk)
                    print(f"Sound Level: {db_level:.2f} dB")

                    # Check if the sound level exceeds the defined thresholds
                    if db_level < THRESHOLD_SAFE:
                        print("Status: Safe")  # Safe sound level
                    elif THRESHOLD_SAFE <= db_level < THRESHOLD_MODERATE:
                        print("Status: Moderate - Be cautious!")  # Moderate sound level
                    else:
                        print("WARNING: Dangerous sound level! Reduce exposure!")  # Dangerous sound level

                except queue.Empty:
                    print("No audio data received. Waiting...")  # Handle if no audio data is available
                    continue

    except KeyboardInterrupt:
        print("\nMonitoring stopped by user.")  # Stop the monitoring if the user interrupts the process

# Run the sound quality monitoring function
monitor_sound_quality()


Monitoring sound levels for 1 minute...
No audio data received. Waiting...
Sound Level: -68.61 dB
Status: Safe
Sound Level: -78.75 dB
Status: Safe
Sound Level: -85.80 dB
Status: Safe
Sound Level: -80.19 dB
Status: Safe
Sound Level: -96.12 dB
Status: Safe
Sound Level: -96.13 dB
Status: Safe
Sound Level: -96.13 dB
Status: Safe
Sound Level: -39.34 dB
Status: Safe
Sound Level: -28.94 dB
Status: Safe
Sound Level: -31.47 dB
Status: Safe
Sound Level: -28.88 dB
Status: Safe
Sound Level: -34.40 dB
Status: Safe
Sound Level: -29.41 dB
Status: Safe
Sound Level: -29.69 dB
Status: Safe
Sound Level: -29.85 dB
Status: Safe
Sound Level: -26.94 dB
Status: Safe
Sound Level: -31.20 dB
Status: Safe
Sound Level: -31.13 dB
Status: Safe
Sound Level: -29.13 dB
Status: Safe
Sound Level: -37.81 dB
Status: Safe
Sound Level: -33.96 dB
Status: Safe
No audio data received. Waiting...
Sound Level: -42.48 dB
Status: Safe
Sound Level: -38.86 dB
Status: Safe
Sound Level: -29.30 dB
Status: Safe
Sound Level: -34.20 dB
Sta